In [ ]:
!nvidia-smi

Mon Oct 31 12:32:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    41W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers evaluate sentencepiece rouge_score sacrebleu -q

In [ ]:
# https://github.com/huggingface/datasets/issues/5117
!pip install -U git+https://github.com/huggingface/datasets -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done




---



In [ ]:
from datasets import load_dataset

In [ ]:
ds = load_dataset("dominguesm/positive-reframing-ptbr-dataset")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['original_text', 'reframed_text', 'strategy', 'strategy_original_text'],
        num_rows: 6679
    })
    dev: Dataset({
        features: ['original_text', 'reframed_text', 'strategy', 'strategy_original_text'],
        num_rows: 835
    })
    test: Dataset({
        features: ['original_text', 'reframed_text', 'strategy', 'strategy_original_text'],
        num_rows: 835
    })
})



---



In [ ]:
ptt5_id = "unicamp-dl/ptt5-base-portuguese-vocab"

In [ ]:
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(ptt5_id)

In [ ]:
def preprocess_function(examples):

    inputs = examples["strategy_original_text"]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples["reframed_text"], max_length=1024, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_ds = ds.map(preprocess_function, batched=True, load_from_cache_file=False)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]



---



In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from transformers import DataCollatorForSeq2Seq

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(ptt5_id)

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy = "steps",
    load_best_model_at_end=True,
    metric_for_best_model = "loss",
    greater_is_better = False,
    save_steps = 500,
    learning_rate=1e-4,
    gradient_accumulation_steps = 2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=48,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    do_train=True,
    do_eval=True,
    fp16=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: strategy_original_text, reframed_text, original_text, strategy. If strategy_original_text, reframed_text, original_text, strategy are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6679
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 3120
You're using a T5TokenizerFast tokenizer. Please note that with a fast tok

Step,Training Loss,Validation Loss
500,2.602200,2.402770
1000,2.112300,2.393514
1500,1.896500,2.418710
2000,1.750900,2.473314
2500,1.657100,2.491197
3000,1.604600,2.502053


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: strategy_original_text, reframed_text, original_text, strategy. If strategy_original_text, reframed_text, original_text, strategy are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 835
  Batch size = 48
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Copy vocab file to ./results/checkpoint-500/spiece.model
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: strategy_original_text, refr

TrainOutput(global_step=3120, training_loss=1.9239707213181716, metrics={'train_runtime': 963.5381, 'train_samples_per_second': 207.952, 'train_steps_per_second': 3.238, 'total_flos': 1.700584606522368e+16, 'train_loss': 1.9239707213181716, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: strategy_original_text, reframed_text, original_text, strategy. If strategy_original_text, reframed_text, original_text, strategy are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 835
  Batch size = 48


{'eval_loss': 2.3935139179229736,
 'eval_runtime': 0.9252,
 'eval_samples_per_second': 902.491,
 'eval_steps_per_second': 19.455,
 'epoch': 30.0}

In [ ]:
trainer.save_model("output/reframer")

Saving model checkpoint to output/reframer
Configuration saved in output/reframer/config.json
Model weights saved in output/reframer/pytorch_model.bin
tokenizer config file saved in output/reframer/tokenizer_config.json
Special tokens file saved in output/reframer/special_tokens_map.json
Copy vocab file to output/reframer/spiece.model




---



In [ ]:
from transformers import pipeline

In [ ]:
reframer = pipeline('summarization', "output/reframer")

loading configuration file output/reframer/config.json
Model config T5Config {
  "_name_or_path": "output/reframer",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "use_cache": true,
  "vocab_size": 32128
}

loading configuration file output/reframer/config.json
Model config T5Config {
  "_name_or_path": "output/reframer",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_f

In [ ]:
input_text = "['growth', 'neutralizing']: Sempre estressado e pensando em um monte de coisas ao mesmo tempo, preciso levar uma de cada vez, sobrecarga estressada, necessidade de reclamar."

In [ ]:
reframer(input_text, max_length=1024)

Your max_length is set to 1024, but you input_length is only 53. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)


[{'summary_text': 'Eu tenho que ser mais produtivo e não reclamar.'}]